In [ ]:
#import statements
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from helper_methods import print_schema, encode_numeric_zscore, encode_text_dummy, print_column, encode_text_index, to_xy, plot_losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
import matplotlib.pyplot as plt 

### Setup Environment
Create folders for putting test output in

In [ ]:
base_path=os.path.join(os.getcwd(), 'test-output/')

iteration='iteration-1'
full_path = os.path.join(base_path, iteration)
try:
        os.mkdir(base_path)
except Exception as e:
     print(f"Probably safe to ignore the following error: \n{e}")
try:
    os.mkdir(full_path)
except Exception as e:
    print(f"{e}")

In [ ]:
#read from train/test datasets
training_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_training-set.csv'))
testing_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_testing-set.csv'))

In [ ]:
#drop all entries that have a missing value anywhere in the set
training_dataframe = training_dataframe.replace('-', np.nan).dropna()
testing_dataframe = testing_dataframe.replace('-', np.nan).dropna()


In [ ]:
testing_dataframe.shape

In [ ]:
#get one instance of each value from each dataframe
training_unique = training_dataframe["proto"].unique()
testing_unique = testing_dataframe["proto"].unique()

#get only values from each numpy array
common_values = np.intersect1d(training_unique, testing_unique)

#make a filtered dataframe containing only values in common_values
training_dataframe = training_dataframe[training_dataframe['proto'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['proto'].isin(common_values)]

#same for service
training_unique = training_dataframe["service"].unique()
testing_unique = testing_dataframe["service"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['service'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['service'].isin(common_values)]

#same for state
training_unique = training_dataframe["state"].unique()
testing_unique = testing_dataframe["state"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['state'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['state'].isin(common_values)]


In [ ]:
testing_dataframe.shape

In [ ]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

### Encode Data

One-hot encode relevant rows


In [ ]:
# one-hot encode non-y features of training dataset
encode_text_dummy(training_dataframe, 'proto')
encode_text_dummy(training_dataframe, 'service')
encode_text_dummy(training_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_index(training_dataframe, 'attack_cat')
# one-hot encode non-y features of testing dataset
encode_text_dummy(testing_dataframe, 'proto')
encode_text_dummy(testing_dataframe, 'service')
encode_text_dummy(testing_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_index(testing_dataframe, 'attack_cat')


In [ ]:
print_schema(training_dataframe)
print_schema(testing_dataframe)


Z-Score relevant rows

In [ ]:
for column in training_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(training_dataframe, column)
for column in testing_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(testing_dataframe, column)


In [ ]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

In [ ]:
x_train,y_train=to_xy(training_dataframe, 'label')
x_test, y_test=to_xy(testing_dataframe, 'label')

### Create Fully Connected Model
Design the model's architecture. 

In [ ]:
# TODO implement a more robust training setup, implement model tabulation 

checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "best_weights.keras"), verbose=0, save_best_only=True) # save best model
for i in range(5):
        print(f"training model: {i}")
        model = Sequential()
        model.add(Dense(512, input_dim=x_train.shape[1], activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        # model.add(Dense(4, activation='relu'))
        model.add(Dense(2, activation='sigmoid')) # single output w/o activation function b/c regression
        model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.01))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=2, mode='min', restore_best_weights=True)
        history = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=64, callbacks=[monitor, checkpointer], verbose=2, epochs=100)
        plot_losses(history, full_path, i)
# load model, do prediction, do evaulation
model.load_weights(os.path.join(full_path, "best_weights.keras"))
prediction = model.predict(x_test)